## Question 1. ZLG algorithm implementation (50 points)
**You are to implement the ZLG algorithm for this problem.**
- **We will use a subset of multiclass data where the label is a protein subcellular localization.**
- **The 8 features are extracted from the protein sequence.**
- **For this problem we are only using points with labels `MIT` or `NUC`.**
- **A total of 892 data points have labels `MIT` (244) or `NUC` (429). We start with the labels of only the first 200 data points (set `Y_k`). The other 792 points are in `Y_u`.**

**First, read the paper and answer the following questions.**
#### 1. What is the idea behind the ZLG algorithm (5 points)?
#### 2. What are the assumptions behind the ZLG algorithm (5 points)?
#### 3. What are the pros and cons of the ZLG algorithm (5points)?

### Imports

In [1]:
import numpy as np
import pandas as pd 
from scipy.spatial import distance_matrix
from sklearn.preprocessing import LabelEncoder

### Data Prep

In [12]:
data = pd.read_csv('data/data.csv')
print('shape',data.shape)
print('unique labels', data.Label.unique())
data.head(5)

shape (673, 9)
unique labels ['MIT' 'NUC']


,X1,X2,X3,X4,X5,X6,X7,X8,Label
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [17]:
# filter out records without desired labels
data_MITNUC = data.loc[data['Label'].isin(['MIT','NUC'])].values

# split data into features and target, encode target classes as 0 or 1
X = data_MITNUC[:,:8]
y = LabelEncoder().fit_transform(data_MITNUC[:,-1])

print(X.shape)
print(y.shape)

(673, 8)
(673,)


In [18]:
# split data into first 200 records and remainder
n_l = 200

Xk = X[:n_l,:]
Yk = y[:n_l]
Xu = X[n_l:,:]
Yu = y[n_l:]

print(Xk.shape)
print(Yk.shape)
print(Xu.shape)
print(Yu.shape)

(200, 8)
(200,)
(473, 8)
(473,)


# 1.1. Part 1 (5 points)
**TODO:**
- **Let's first construct the weight matrix W.**
- **Use $t = 0$ and $\sigma$ as the standard deviation of $X$.**
- **Then calculate the $D$ matrix and the Laplacian matrix (Delta).**

### Formulas
Similarity is measured using the radial basis function (RBF):

$$w_{ij}=\exp{\left( -\sum_{d=1}^m \frac{(x_{id} - x_{jd})^2}{\sigma^2_d} \right)}$$










where
- $x_i \in \mathbb{R}^m$
- $x_{id}$ is the $d$-th component  of instance $x_i$
- $\sigma_1, \ldots ,\sigma_m$ are length scale hyperparameters for each dimension


Note: $\sum_{d=1}^m \left( x_{id} - x_{jd} \right)^2$ is the squared Euclidean distance between $x_i$ and $x_j$

In [29]:
# other paper
# https://www.aaai.org/Papers/ICML/2003/ICML03-118.pdf

In [ ]:
def Laplacian_matrix(X):
    ## TODO ##
    
    return Delta

Delta = Laplacian_matrix(X)

# 1.2. Part 2 (5 points) 
**TODO:**
- **Now complete the subroutine to compute the minimum-energy solution for the unlabeled instances. (Hint: Use the formula in page 38, Lecture 7.)** 
- **The function also outputs one submatrix that we will use to select points to query.**

In [ ]:
def minimum_energy_solution(Delta,n_l,fl):
    """
    Args:
        Delta: The Laplacian matrix. 
        n_l: Number of labeled points. Notice that Delta should have the upper left submatrix 
            corresponding to these n_l points. So when new points get labeled, you may need 
            to rearrange the matrix.
        fl: Known labels.
    Returns:
        Delta_uu_inv: Inverse matrix of the submatrix corresponding to unlabeled points.
        fu: Minimum energy solution of all unlabeled points.
    """
    ## TODO ##
    
    return Delta_uu_inv, fu

Delta_uu_inv, fu = minimum_energy_solution(Delta,n_l,Yk)


# 1.3. Part 3 (15 points) 
**TODO:**
- **We would like to query the points that minimize the expected risk. To do so, we want to be able to calculate the expected estimated risk after querying any point $k$.**
- **The variable `Rhat_fplus_xk` refers to $\hat{R}(f^{+x_k})$.**
- **`fu_xk0` is $f_u^{+(x_k,0)}$ and vice versa for `fu_xk1`.**

I'm confused about the notation involved in calculating expected risk for ZLG. In the paper, we have the following equations:

#### Expected risk:

$$\hat{R}\left( f^{+(x_k,y_k)} \right)=\sum_{i=1}^n min\left( f_i^{+(x_k,y_k)},1-f_i^{+(x_k,y_k)}\right)$$

#### Expected estimated risk:

$$\hat{R}\left( f^{+x_k} \right)=
(1-f_k)\hat{R}\left( f^{+(x_k,0)} \right)
+   f_k\hat{R}\left( f^{+(x_k,1)} \right)$$


#### Conditional Distribution of all unlabeled nodes:

$$f_u^{+(x_k,y_k)}=f_u+(y_k-f_k)\frac{(\Delta_{uu}^{-1})_{ \cdot k}}{(\Delta_{uu}^{-1})_{kk}}$$


***

I'm confused at the difference between $f_u$ and $f$ and $f_i$. The paper defines $f=\begin{bmatrix}f_l \\ f_u\end{bmatrix}$ but I don't understand how exactly this works through the equations above. My understanding is that $f_u$ is a vector. I calculated that $f_u^{+(x_k,0)}$ is a vector of the same dimension.  

Is $f^{+(x_k,0)}$ the same as $f_u^{+(x_k,0)}$?

***

I'm also confused at how to calculate estimated risk.

$$\hat{R}\left( f^{+(x_k,0)} \right)=\sum_{i=1}^n min\left( f_i^{+(x_k,0)},1-f_i^{+(x_k,0)}\right)$$

What is the $n$ in this case? I thought we were dealing with $f_u$. What would $f_1, f_2, \ldots, f_n$ be?

***







In [ ]:
def expected_estimated_risk(Delta_uu_inv,k,fu):
    """
    Args:
        Delta_uu_inv: Inverse matrix of the submatrix corresponding to unlabeled points.
        k: index of one unlabeled point with respect to the uu submatrix (not the entire Delta)
        fu: Minimum energy solution of all unlabeled points.
    Returns:
        Rhat_fplus_xk: Expected estimated risk after querying node k
    """
    ## fu plus xk, yk = 0
    fu_xk0 = fu + (0 - fu[k])*Delta_uu_inv[:,k]/Delta_uu_inv[k,k]
    ## fu plus xk, yk = 1
    fu_xk1 = fu + (1 - fu[k])*Delta_uu_inv[:,k]/Delta_uu_inv[k,k]
    
    ## TODO ##
    
    return Rhat_fplus_xk
        

# 1.4. Part 4 (5 points) 
**TODO:**
- **Compute the above expected estimated risk for all unlabeled points and select one to query.**
- **Let's try query 100 points. Which points are queried? Compare with random queries and make a plot.**

In [ ]:
def zlg_query(Delta_uu_inv,n_l,fu,n_samples):
    """
    Args:
        Delta_uu_inv: Inverse matrix of the submatrix corresponding to unlabeled points.
        n_l: Number of labeled points.
        fu: Minimum energy solution of all unlabeled points.
        n_samples: Number of samples.
    Returns:
        query_idx: the idx of the point to query, wrt the unlabeled points 
                (idx is 0 if it's the first unlabeled point)
    """
    n_u = n_samples - n_l
    query_idx = 0
    min_Rhat = np.inf
    ## TODO ##
    
    return query_idx

In [ ]:
n_samples = X.shape[0]
for t in range(100):
    ## edit this block ##
    query_idx = zlg_query(Delta_uu_inv,n_l,fu,n_samples)
    Yk = np.append(Yk,Yu[query_idx])
    Yu = np.delete(Yu,query_idx)
    Xk = np.append(Xk,[Xu[query_idx,:]],axis=0)
    Xu = np.delete(Xu,query_idx, 0)
    n_l += 1
    Delta = Laplacian_matrix(np.concatenate((Xk,Xu),axis=0))
    Delta_uu_inv, fu = minimum_energy_solution(Delta,n_l,Yk)
    print(query_idx)
    ## TODO ##

# 1.5. Bonus question 

**Answer the following questions. (Your grade will not exceed 100 for this homework.)**

#### 1. For this dataset, how many labeled data points do you actually need to train the model sufficiently well? 
#### 2. And why?